In [1]:
using CSV, DataFrames, Statistics

In [2]:
function method_category(met)
    if startswith(meth, "Imp-then-Reg")
        return "Imp-then-Reg"
    elseif startswith(meth, "Joint Imp-then-Reg")
        return "Joint Imp-then-Reg"
    elseif meth ∈ ["Static", "Affine", "Finite"]
        return "Adaptive LR"
    elseif startswith(meth, "Complete Features")
        return "Complete Features"
    else 
        return meth
    end
end

method_category (generic function with 1 method)

In [3]:
pb_datasets = ["cylinder-bands", "ozone-level-detection-eight", "ozone-level-detection-one", "thyroid-disease-thyroid-0387", "trains"]

5-element Vector{String}:
 "cylinder-bands"
 "ozone-level-detection-eight"
 "ozone-level-detection-one"
 "thyroid-disease-thyroid-0387"
 "trains"

## For Real X - Syn Y Experiments

In [ ]:
setting = "fakey/"
for dir = ["tree_mar/", "tree_nmar/", "tree_mar_adv/"]
    directory = setting*dir
    subdir = "final/"
    
    filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv")]
    res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
    for i in 1:length(filelist)
        res = vcat(res, CSV.read(directory*subdir*filelist[i], DataFrame))
    end

    res[!,:method_cat] = map(t -> method_category, res[:,:method])
    res[!,:X_setting] .= "real_X_"*split(dir, "_")[2][1:end-1]
    res[!,:Y_setting] .= "syn_Y_"*split(dir, "_")[1]

    CSV.write(directory*"FINAL_results.csv", res)
end

In [ ]:
res = vcat(CSV.read(setting*"tree_mar/"*"FINAL_results.csv", DataFrame),
    CSV.read(setting*"tree_nmar/"*"FINAL_results.csv", DataFrame),
    CSV.read(setting*"tree_mar_adv/"*"FINAL_results.csv", DataFrame))

In [ ]:
filter!(t -> t[:dataset] ∉ pb_datasets, res)

In [ ]:
for method in ["Complete Features", "Imp-then-Reg 1", "Imp-then-Reg 2", "Imp-then-Reg 3", "Imp-then-Reg 4", "Joint Imp-then-Reg"]
    aux = filter(t -> startswith(t[:method], method), res)
    # @show size(aux)
    idcols = [:dataset, :X_setting, :Y_setting, :SNR, :k, :kMissing, :splitnum]
    gd = groupby(aux, idcols)

    aux = similar(aux, 0)
    for subdf in gd 
        scoremax = argmax(subdf[:,:score])
        push!(aux, subdf[scoremax,names(aux)])
    end
    aux[!,:method] .= method*" - best"

    res = vcat(res, aux)
end

In [ ]:
CSV.write(setting*"tree_mar/"*"FINAL_TREE_results.csv", res)

In [ ]:
res = vcat(CSV.read(setting*"tree_mar/"*"FINAL_results.csv", DataFrame),
    CSV.read(setting*"tree_nmar/"*"FINAL_results.csv", DataFrame),
    CSV.read(setting*"tree_mar_adv/"*"FINAL_results.csv", DataFrame))

filter!(t -> t[:dataset] ∉ pb_datasets, res)

In [ ]:
gd = groupby(unique(res[:,[:dataset, :X_setting, :kMissing, :splitnum]]), 
    [:dataset, :X_setting, :kMissing])
unique(
    innerjoin(res, 
        filter(t -> t[:nrow] < 10, combine(gd, nrow)), on = [:dataset, :X_setting, :kMissing])[:,[:dataset, :X_setting, :kMissing]])

In [ ]:
check = leftjoin(combine(groupby(unique(res[:,[:dataset,:X_setting, :kMissing]]), [:dataset,:X_setting]), nrow), 
    df, 
    on =:dataset)
check[(check[:,:nrow] .< check[:,:p_miss_num] .+ 1),:]

In [ ]:
dataset_list = [d for d in readdir("../datasets/") if !startswith(d, ".")]
sort!(dataset_list)
redolist = []            
for d in unique(check[(check[:,:nrow] .< check[:,:p_miss_num] .+ 1),:dataset]) # unique(filter(t -> t[:nrow] < 10, combine(gd, nrow))[:,:dataset])
    @show d, findfirst(dataset_list .== d) - 1
                push!(redolist, findfirst(dataset_list .== d) - 1)
end

In [ ]:
4,5,10,21,23,24,31,38,43,49,61,62,63,64,65,66,67,68,70

In [ ]:
df1 = CSV.read("pattern_counts_allfeat.csv", DataFrame) #Before one-hot-encoding
select!(df1, [:Name, :n, :p, :Num_Patterns, :p_miss])
rename!(df1, :p => :p_all, :p_miss => :p_miss_all)
df2 = CSV.read("pattern_counts_numonly.csv", DataFrame) #After one-hot-encoding
select!(df2, [:Name, :p, :p_miss])
rename!(df2, :p => :p_num, :p_miss => :p_miss_num)
df = innerjoin(df1, df2, on=:Name, makeunique=true)
df[!,:missing_num] .= df[:,:p_miss_num]
df[!,:missing_cat] .= df[:,:p_miss_all] .- df[:,:p_miss_num]
rename!(df, :Name => :dataset)
select!(df, [:dataset, :missing_cat, :p_miss_all, :p_miss_num, :p_num])

## For Real Data Experiments

In [ ]:
for directory = ["realy/"]
    subdir="2022-08-23/"
    filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
    res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
    for i in 1:length(filelist)
        res = vcat(res, CSV.read(directory*subdir*filelist[i], DataFrame))
    end
    # filter!(t -> t[:k] > 0, res) #Remove dataset with only a bias term
    res[!,:method_cat] = map(t -> method_category, res[:,:method])
    res[!,:X_setting] .= "real_X"
    res[!,:Y_setting] .= "real_Y"
    CSV.write(directory*"FINAL_results.csv", res)
end

Sanity check

In [ ]:
res = CSV.read("realy/"*"FINAL_results.csv", DataFrame)

In [ ]:
unique(filter( t-> t[:nrow] < 10, combine(groupby(res, [:dataset, :method]), nrow))[:,[:dataset, :nrow]])

In [ ]:
res = CSV.read("realy/"*"FINAL_results.csv", DataFrame)

In [ ]:
filter!(t -> t[:dataset] ∉ pb_datasets, res)

In [ ]:
for method in ["Complete Features", "Imp-then-Reg 1", "Imp-then-Reg 2", "Imp-then-Reg 3", "Imp-then-Reg 4", "Joint Imp-then-Reg"]
    aux = filter(t -> startswith(t[:method], method), res)
    # @show size(aux)
    idcols = [:dataset, :SNR, :k, :kMissing, :splitnum]
    gd = groupby(aux, idcols)

    aux = similar(aux, 0)
    for subdf in gd 
        scoremax = argmax(subdf[:,:score])
        push!(aux, subdf[scoremax,names(aux)])
    end
    aux[!,:method] .= method*" - best"

    res = vcat(res, aux)
end

In [ ]:
CSV.write("realy/"*"FINAL_results.csv", res)

In [ ]:
names(res)

# For Synthetic-Data Experiments

In [4]:
1+1

2

In [5]:
for dir = ["linear_mar/", "linear_censoring/", "nn_mar/", "nn_censoring/", "tree_mar/", "tree_censoring/"]
    directory = "synthetic/"*dir
    subdir="final/"
    
    filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
    res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
    for i in 1:length(filelist)
        aux = CSV.read(directory*subdir*filelist[i], DataFrame)
        if any(aux[:,:pMissing] .> 0)
            missingproba = unique(aux[aux[:,:pMissing] .> 0,:pMissing])[1]
            aux[!,:pMissing] .= missingproba
        end
        try
            res = vcat(res, aux)
        catch 
            println("Error with ", directory*subdir*filelist[i])
        end
    end
    # filter!(t -> t[:k] > 0, res) #Remove dataset with only a bias term
    res[!,:method_cat] = map(t -> method_category, res[:,:method])
    res[!,:X_setting] .= "syn_X_"*split(dir, "_")[2][1:end-1]
    res[!,:Y_setting] .= "syn_Y_"*split(dir, "_")[1]

    CSV.write(directory*"FINAL_results.csv", res)
end

Sanity check

In [ ]:
df = CSV.read("synthetic/linear_mar/FINAL_results.csv", DataFrame) 
df[!,:setting] .= "1 - Lin-MAR"

aux = CSV.read("synthetic/linear_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "2 - Lin-NMAR"
df = vcat(df, aux)
 
aux = CSV.read("synthetic/tree_mar/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "3 - Tree-MAR"
df = vcat(df, aux)

aux = CSV.read("synthetic/tree_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "4 - Tree-NMAR"
df = vcat(df, aux)

aux = CSV.read("synthetic/nn_mar/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "5 - NN-MAR"
df = vcat(df, aux)

aux = CSV.read("synthetic/nn_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "6 - NN-NMAR"
df = vcat(df, aux)

In [ ]:
combine(groupby(df, [:dataset, :method, :setting]), nrow)

In [ ]:
unique(combine(groupby(df, [:dataset, :method, :setting]), nrow)[:,:nrow])

In [ ]:
filter( t-> t[:nrow] < 90, combine(groupby(res, [:dataset, :method, :setting]), nrow))

In [ ]:
pbdata = unique(filter( t -> t[:nrow] < 90, combine(groupby(df, [:dataset, :method, :setting]), nrow))[:,[:method, :setting, :dataset]])

In [ ]:
aux = filter( t -> t[:dataset] ∈ pbdata && t[:nrow] < 10, combine(groupby(df, [:dataset, :method, :setting, :pMissing]), nrow))
unique(aux[:,[:dataset, :setting, :pMissing, :nrow]])

In [ ]:
filter(t -> t[:pMissing] == 0.1, unique(aux[:,[:dataset, :setting, :pMissing, :nrow]]))

In [ ]:
filter(t -> t[:pMissing] == 0.3, unique(aux[:,[:dataset, :setting, :pMissing, :nrow]]))

In [ ]:
filter(t -> startswith(t[:setting], "5"), unique(aux[:,[:dataset, :setting, :pMissing, :nrow]]))